## 1.Table 2: Average performance  and [95% confidence intervals] for logistic regression models using ICD Only, Med Only, Note Only, and ICD+MED+Note in the MGH+BIDMC training sets and MGH+BIDMC testing sets

### 1.1  Note **Only**

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
)
from sklearn.compose import ColumnTransformer
np.random.seed(2025)

# Load training and test datasets
train_data = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/mgh_bidmc_3948.csv')  # Training dataset
test_data = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/mgh_bidmc_1664.csv')  # Test dataset

# Define the text feature extractor using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))  # Extract unigrams, bigrams, and trigrams

# Define class weights to handle imbalanced classes
class_weight_dict = {0: 1.0, 1: 3.0}  # Assign higher weight to the positive class

# Define the classification model
logistic_model = LogisticRegression(solver='liblinear', class_weight=class_weight_dict, random_state=2025)  # Logistic regression with weighted classes

# Define the preprocessor for feature extraction
data_preprocessor = ColumnTransformer([
    ('tfidf', tfidf_vectorizer, 'report_text'),  # Apply TF-IDF to the 'report_text' column
], n_jobs=-1)

# Create a pipeline for preprocessing and classification
classification_pipeline = Pipeline([
    ('preprocessor', data_preprocessor),
    ('classifier', logistic_model)
])

# Define the parameter grid for hyperparameter tuning
hyperparameter_grid = {
    'classifier__penalty': ['l1'],  # L1 regularization
    'classifier__C': [0.01, 0.1, 1.0, 10.0]  # Regularization strength
}

# Perform grid search for hyperparameter optimization
grid_search = GridSearchCV(classification_pipeline, param_grid=hyperparameter_grid, cv=5, n_jobs=-1)  # 5-fold cross-validation
grid_search.fit(train_data[['report_text']], train_data['annot'])  # Fit the model on the training data

# Display the best parameters and the corresponding score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

# Evaluate the model on the test dataset
test_features = test_data[['report_text']]  # Features from the test set
test_labels = test_data['annot']  # Labels from the test set
test_predictions = grid_search.predict(test_features)  # Predict labels for the test set

# Perform bootstrapping on the test set to calculate evaluation metrics
bootstrap_iterations = 10  # Number of bootstrap iterations
evaluation_metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'specificity': [],
    'roc_auc': [],
    'auprc': []
}

for _ in range(bootstrap_iterations):
    # Sample test set with replacement
    sampled_indices = np.random.choice(len(test_features), len(test_features), replace=True)
    sampled_features = test_features.iloc[sampled_indices]
    sampled_labels = test_labels.iloc[sampled_indices]

    # Predict on the sampled data
    sampled_predictions = grid_search.predict(sampled_features)
    sampled_probabilities = grid_search.predict_proba(sampled_features)[:, 1]  # Probability for the positive class

    # Calculate evaluation metrics
    tn, fp, fn, tp = confusion_matrix(sampled_labels, sampled_predictions).ravel()
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    evaluation_metrics['accuracy'].append(accuracy_score(sampled_labels, sampled_predictions))
    evaluation_metrics['precision'].append(precision_score(sampled_labels, sampled_predictions))
    evaluation_metrics['recall'].append(recall_score(sampled_labels, sampled_predictions))
    evaluation_metrics['f1'].append(f1_score(sampled_labels, sampled_predictions))
    evaluation_metrics['specificity'].append(specificity)
    evaluation_metrics['roc_auc'].append(roc_auc_score(sampled_labels, sampled_probabilities))
    evaluation_metrics['auprc'].append(average_precision_score(sampled_labels, sampled_probabilities))

# Calculate the mean and 95% confidence interval for each metric
for metric_name, metric_values in evaluation_metrics.items():
    mean_value = np.mean(metric_values)
    lower_bound = np.percentile(metric_values, 2.5)  # 2.5th percentile
    upper_bound = np.percentile(metric_values, 97.5)  # 97.5th percentile

    print(f"{metric_name.capitalize()}: Mean={mean_value}, 95% CI=({lower_bound}, {upper_bound})")


Best Parameters:  {'classifier__C': 1.0, 'classifier__penalty': 'l1'}
Best Score:  0.915924981149027
Accuracy: Mean=0.8936899038461539, 95% CI=(0.8818209134615385, 0.9088942307692308)
Precision: Mean=0.8886770498364014, 95% CI=(0.8731285665127749, 0.9071458351201197)
Recall: Mean=0.8855674029265812, 95% CI=(0.8687843481230788, 0.9054281655844156)
F1: Mean=0.887080392028369, 95% CI=(0.8728417899929528, 0.9042928173835972)
Specificity: Mean=0.900875676281894, 95% CI=(0.8849602957105012, 0.9145477658135664)
Roc_auc: Mean=0.9505194139312042, 95% CI=(0.942088730768383, 0.9600750052684119)
Auprc: Mean=0.9495666842376862, 95% CI=(0.941345046239817, 0.9581640617200945)


Best Parameters:  {'classifier__C': 1.0, 'classifier__penalty': 'l1'}
Best Score:  0.915924981149027
Accuracy: Mean=0.8903846153846156, 95% CI=(0.8792067307692307, 0.9003004807692307)
Precision: Mean=0.8878943770543042, 95% CI=(0.8690930397933417, 0.9027105702074429)
Recall: Mean=0.8783526438224714, 95% CI=(0.8684446424827008, 0.8997620545864902)
F1: Mean=0.8830274360263596, 95% CI=(0.8718387280531478, 0.891931275914624)
Specificity: Mean=0.9011045991650632, 95% CI=(0.8833159103394945, 0.919364645632073)
Roc_auc: Mean=0.9467935690783327, 95% CI=(0.9407033765153447, 0.9520673768926069)
Auprc: Mean=0.9445529954909692, 95% CI=(0.9367879638540018, 0.949813157060619)

### 1.2 ICD+MED+Note

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Load training and test datasets
train_data = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/mgh_bidmc_3948.csv')
test_data = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/mgh_bidmc_1664.csv')

# Define the text feature extractor using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))

# Define class weights to handle imbalanced classes
class_weights = {0: 1.0, 1: 3.0}

# Define the classification model
logistic_model = LogisticRegression(solver='liblinear', class_weight=class_weights, random_state=2025)  # Logistic regression with weighted classes

# Define the preprocessor for feature extraction and scaling
preprocessor = ColumnTransformer([
    ('text_vectorizer', tfidf_vectorizer, 'report_text'),  # Apply TF-IDF to 'report_text'
    ('scaler', StandardScaler(), ['icd', 'med'])  # Scale numerical features 'icd' and 'med'
], n_jobs=-1)

# Create a pipeline for preprocessing and classification
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', logistic_model)
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'classifier__penalty': ['l1'],  # L1 regularization
    'classifier__C': [0.01, 0.1, 1.0, 10.0]  # Regularization strength
}

# Perform grid search for hyperparameter optimization
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1)  # 5-fold cross-validation
grid_search.fit(train_data[['icd', 'med', 'report_text']], train_data['annot'])  # Fit on training data

# Display the best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

# Evaluate the model on the test dataset
X_test = test_data[['icd', 'med', 'report_text']]
y_test = test_data['annot']
y_pred = grid_search.predict(X_test)

# Perform bootstrapping on the test set to calculate evaluation metrics
n_iterations = 10  # Number of bootstrap iterations
metrics_values = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'specificity': [],
    'roc_auc': [],
    'auprc': []
}

for _ in range(n_iterations):
    # Sample test set with replacement
    sampled_indices = np.random.choice(len(X_test), len(X_test), replace=True)
    X_sampled = X_test.iloc[sampled_indices]
    y_sampled = y_test.iloc[sampled_indices]

    # Predict on the sampled data
    y_pred_sampled = grid_search.predict(X_sampled)
    y_pred_prob_sampled = grid_search.predict_proba(X_sampled)[:, 1]  # Probability of the positive class

    # Calculate metrics
    tn, fp, fn, tp = confusion_matrix(y_sampled, y_pred_sampled).ravel()
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    metrics_values['accuracy'].append(accuracy_score(y_sampled, y_pred_sampled))
    metrics_values['precision'].append(precision_score(y_sampled, y_pred_sampled))
    metrics_values['recall'].append(recall_score(y_sampled, y_pred_sampled))
    metrics_values['f1'].append(f1_score(y_sampled, y_pred_sampled))
    metrics_values['specificity'].append(specificity)
    metrics_values['roc_auc'].append(roc_auc_score(y_sampled, y_pred_prob_sampled))
    metrics_values['auprc'].append(average_precision_score(y_sampled, y_pred_prob_sampled))

# Calculate mean and 95% confidence intervals for each metric
for metric, values in metrics_values.items():
    mean_value = np.mean(values)
    lower_bound = np.percentile(values, 2.5)
    upper_bound = np.percentile(values, 97.5)

    print(f"{metric.capitalize()}: Mean={mean_value:.4f}, 95% CI=({lower_bound:.4f}, {upper_bound:.4f})")


Best Parameters:  {'classifier__C': 1.0, 'classifier__penalty': 'l1'}
Best Score:  0.8941525083826667
Accuracy: Mean=0.8917, 95% CI=(0.8834, 0.9032)
Precision: Mean=0.8885, 95% CI=(0.8752, 0.8964)
Recall: Mean=0.8774, 95% CI=(0.8565, 0.8979)
F1: Mean=0.8829, 95% CI=(0.8739, 0.8948)
Specificity: Mean=0.9040, 95% CI=(0.8856, 0.9150)
Roc_auc: Mean=0.9489, 95% CI=(0.9421, 0.9545)
Auprc: Mean=0.9470, 95% CI=(0.9388, 0.9531)


Best Parameters:  {'classifier__C': 1.0, 'classifier__penalty': 'l1'}
Best Score:  0.8941525083826667
Accuracy: Mean=0.8882, 95% CI=(0.8797, 0.8982)
Precision: Mean=0.8847, 95% CI=(0.8643, 0.8941)
Recall: Mean=0.8744, 95% CI=(0.8579, 0.8934)
F1: Mean=0.8795, 95% CI=(0.8690, 0.8923)
Specificity: Mean=0.9003, 95% CI=(0.8853, 0.9106)
Roc_auc: Mean=0.9503, 95% CI=(0.9411, 0.9576)
Auprc: Mean=0.9493, 95% CI=(0.9440, 0.9554)

### 1.3 ICD Only

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, auc, precision_recall_curve, roc_curve, confusion_matrix
)
from imblearn.over_sampling import RandomOverSampler
from sklearn.base import clone
from scipy.stats import bootstrap

# Load and preprocess the dataset (ensure correct path and format)
df = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/merged_mgh_bidmc222.csv')

# Extract features (X) and target variable (y)
X = df[['icd']]  # ICD codes column
y = df['annot']  # Target variable

#Handle class imbalance with RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Initialize K-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10 splits with shuffling
model = LogisticRegression(class_weight='balanced', random_state=2025)  # Logistic regression model with balanced class weights

# Lists to store results for each fold
accuracies, f1_scores, recalls, precisions = [], [], [], []
aurocs, auprcs, specificities = [], [], []

# Cross-validation loop
for train_index, test_index in kf.split(X_resampled):
    # Split the resampled data into training and test sets
    X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_test = y_resampled.iloc[train_index], y_resampled.iloc[test_index]

    # Clone the model to ensure independence for each fold
    model_clone = clone(model)
    model_clone.fit(X_train, y_train)  # Train the model

    # Make predictions
    y_pred = model_clone.predict(X_test)
    y_pred_proba = model_clone.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Calculate evaluation metrics
    accuracies.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))
    precisions.append(precision_score(y_test, y_pred))

    # ROC and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    aurocs.append(auc(fpr, tpr))

    # Precision-Recall curve and AUPRC
    precision_points, recall_points, _ = precision_recall_curve(y_test, y_pred_proba)
    auprcs.append(auc(recall_points, precision_points))

    # Specificity (True Negative Rate)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))

# Function to compute 95% confidence intervals using bootstrap
def calculate_ci(data):
    result = bootstrap((np.array(data),), np.mean, confidence_level=0.95)
    return result.confidence_interval

# Print confidence intervals for each metric
print("Accuracy CI:", calculate_ci(accuracies))
print("F1-score CI:", calculate_ci(f1_scores))
print("Recall CI:", calculate_ci(recalls))
print("Precision CI:", calculate_ci(precisions))
print("AUROC CI:", calculate_ci(aurocs))
print("AUPRC CI:", calculate_ci(auprcs))
print("Specificity CI:", calculate_ci(specificities))

# Print average values for each metric
print("Average Accuracy:", np.mean(accuracies))
print("Average F1-score:", np.mean(f1_scores))
print("Average Recall:", np.mean(recalls))
print("Average Precision:", np.mean(precisions))
print("Average AUROC:", np.mean(aurocs))
print("Average AUPRC:", np.mean(auprcs))
print("Average Specificity:", np.mean(specificities))


Accuracy CI: ConfidenceInterval(low=np.float64(0.5249661559668469), high=np.float64(0.5526510480887794))
F1-score CI: ConfidenceInterval(low=np.float64(0.5879624029643268), high=np.float64(0.6195655503187343))
Recall CI: ConfidenceInterval(low=np.float64(0.6896647701173418), high=np.float64(0.7190944242192289))
Precision CI: ConfidenceInterval(low=np.float64(0.511418720159229), high=np.float64(0.545396658106908))
AUROC CI: ConfidenceInterval(low=np.float64(0.5269855608153995), high=np.float64(0.549233646724235))
AUPRC CI: ConfidenceInterval(low=np.float64(0.6763990425178069), high=np.float64(0.7032895434829793))
Specificity CI: ConfidenceInterval(low=np.float64(0.3542103681354118), high=np.float64(0.3829318379882789))
Average Accuracy: 0.536861686296186
Average F1-score: 0.602855387552034
Average Recall: 0.7039825385220351
Average Precision: 0.5273178096220179
Average AUROC: 0.5365554399448353
Average AUPRC: 0.689491232669027
Average Specificity: 0.36912834136763567


Accuracy CI: ConfidenceInterval(low=0.5251457695580649, high=0.5525849920732782)
F1-score CI: ConfidenceInterval(low=0.5880069800519575, high=0.6204001727018401)
Recall CI: ConfidenceInterval(low=0.6895476219930763, high=0.718962558867317)
Precision CI: ConfidenceInterval(low=0.5112345655621887, high=0.5451753534821403)
AUROC CI: ConfidenceInterval(low=0.5271609031629594, high=0.5496511963309062)
AUPRC CI: ConfidenceInterval(low=0.6771440143635011, high=0.7036065982878332)
Specificity CI: ConfidenceInterval(low=0.35460200634741884, high=0.38311053485680646)
Average Accuracy: 0.536861686296186
Average F1-score: 0.602855387552034
Average Recall: 0.7039825385220351
Average Precision: 0.5273178096220179
Average AUROC: 0.5365554399448353
Average AUPRC: 0.689491232669027
Average Specificity: 0.36912834136763567

### 1.4 Med Only

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, auc, precision_recall_curve, roc_curve, confusion_matrix
)
from imblearn.over_sampling import RandomOverSampler
from sklearn.base import clone
from scipy.stats import bootstrap
df = pd.read_csv('/home/niels/cdac Dropbox/Niels Turley/codes/data/merged_mgh_bidmc222.csv')


# Assume the dataframe 'df' is preloaded and contains the necessary columns
# Replace 'med' with the actual feature column and 'annot' with the target column
features = df[['med']]  # Feature column
target = df['annot']  # Target variable column

# Handle class imbalance using RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
features_resampled, target_resampled = oversampler.fit_resample(features, target)

# Initialize K-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation
logistic_model = LogisticRegression(class_weight='balanced', random_state=2025)  # Logistic regression with balanced class weights

# Lists to store evaluation metrics for each fold
accuracy_scores = []
f1_scores = []
recall_scores = []
precision_scores = []
roc_auc_scores = []
auprc_scores = []
specificity_scores = []

# Perform cross-validation
for train_indices, test_indices in kf.split(features_resampled):
    # Split the data into training and test sets
    X_train, X_test = features_resampled.iloc[train_indices], features_resampled.iloc[test_indices]
    y_train, y_test = target_resampled.iloc[train_indices], target_resampled.iloc[test_indices]

    # Clone the model to ensure independence for each fold
    model_clone = clone(logistic_model)
    model_clone.fit(X_train, y_train)  # Train the model

    # Make predictions
    y_pred = model_clone.predict(X_test)
    y_pred_prob = model_clone.predict_proba(X_test)[:, 1]  # Probability of the positive class

    # Calculate evaluation metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))

    # ROC curve and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc_scores.append(auc(fpr, tpr))

    # Precision-Recall curve and AUPRC
    precision_points, recall_points, _ = precision_recall_curve(y_test, y_pred_prob)
    auprc_scores.append(auc(recall_points, precision_points))

    # Specificity (True Negative Rate)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    specificity_scores.append(specificity)

# Function to calculate 95% confidence intervals using bootstrap
def calculate_ci(metric_list):
    result = bootstrap((np.array(metric_list),), np.mean, confidence_level=0.95)
    return result.confidence_interval

# Print confidence intervals for each metric
print("Accuracy CI:", calculate_ci(accuracy_scores))
print("F1-score CI:", calculate_ci(f1_scores))
print("Recall CI:", calculate_ci(recall_scores))
print("Precision CI:", calculate_ci(precision_scores))
print("ROC AUC CI:", calculate_ci(roc_auc_scores))
print("AUPRC CI:", calculate_ci(auprc_scores))
print("Specificity CI:", calculate_ci(specificity_scores))

# Print average values for each metric
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average F1-score:", np.mean(f1_scores))
print("Average Recall:", np.mean(recall_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average ROC AUC:", np.mean(roc_auc_scores))
print("Average AUPRC:", np.mean(auprc_scores))
print("Average Specificity:", np.mean(specificity_scores))


Accuracy CI: ConfidenceInterval(low=np.float64(0.5517078753209479), high=np.float64(0.5789836241005318))
F1-score CI: ConfidenceInterval(low=np.float64(0.5976816340505097), high=np.float64(0.6297741967963978))
Recall CI: ConfidenceInterval(low=np.float64(0.6756809622385578), high=np.float64(0.7120933404044725))
Precision CI: ConfidenceInterval(low=np.float64(0.5336752476361454), high=np.float64(0.5688721250778443))
ROC AUC CI: ConfidenceInterval(low=np.float64(0.553223091870324), high=np.float64(0.5769324804424608))
AUPRC CI: ConfidenceInterval(low=np.float64(0.6849527009642313), high=np.float64(0.7120827643957545))
Specificity CI: ConfidenceInterval(low=np.float64(0.4127859352732476), high=np.float64(0.44741206519189936))
Average Accuracy: 0.562256503860101
Average F1-score: 0.6129853415798636
Average Recall: 0.6945198438368045
Average Precision: 0.5490123483450897
Average ROC AUC: 0.5621008090933352
Average AUPRC: 0.6980109862405527
Average Specificity: 0.4296817743498657


Accuracy CI: ConfidenceInterval(low=0.5522011079188255, high=0.5792735530615427)
F1-score CI: ConfidenceInterval(low=0.5976484058577213, high=0.629865040069949)
Recall CI: ConfidenceInterval(low=0.676377129174852, high=0.7114721275824842)
Precision CI: ConfidenceInterval(low=0.5335043401507912, high=0.5690265235460552)
ROC AUC CI: ConfidenceInterval(low=0.5531582611884539, high=0.5768187444821447)
AUPRC CI: ConfidenceInterval(low=0.6854884054211079, high=0.7122810645493387)
Specificity CI: ConfidenceInterval(low=0.4126565799296876, high=0.44753190808435445)
Average Accuracy: 0.562256503860101
Average F1-score: 0.6129853415798636
Average Recall: 0.6945198438368045
Average Precision: 0.5490123483450897
Average ROC AUC: 0.5621008090933352
Average AUPRC: 0.6980109862405527
Average Specificity: 0.4296817743498657